<a href="https://colab.research.google.com/github/FatimaEzzedinee/ML-bachelor-course-labs-sp24/blob/main/07_unsupervised_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning SP 2023/2024

- Prof. Cesare Alippi
- Alvise Dei Rossi ([`alvise.dei.rossi@usi.ch`](mailto:alvise.dei.rossi@usi.ch))<br>
- Fatima Ezzeddine ([`fatima.ezzeddine@usi.ch`](mailto:fatima.ezzeddine@usi.ch))<br>
- Alessandro Manenti ([`alessandro.manenti@usi.ch`](mailto:alessandro.manenti@usi.ch))

---
# Lab 07: Unsupervised learning

In this lab, we will see practical applications of unsupervised learning techniques.

We will focus on two main tasks:

1. Clustering: the goal is to group together similar instances togheter into _clusters_. It's a great tool for several ML tasks like data analysis, semi-supervised learning, and more.
2. Dimensionality reduction: reducing the number of features considerably, possibly turning an intractable problem into a tractable one. Furthermore, it can be incredibly useful for visualization purposes for high dimensional data.

We will use two very famous/common datasets to illustrate the main concepts and apply these methods using sklearn implementations:
 - [Iris](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris)
 - [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist)


## 7.1 Clustering the Iris dataset

---

We've seen this dataset for classification tasks also in the previous lab. However, this time we analyze the Iris dataset by considering **only the features**, without the targets (i.e., the classes). Let's start by loading the dataset and getting a sense of it.

In [ ]:
from sklearn.datasets import load_iris

# The Iris dataset is easily available with sklearn
iris = load_iris()

# several attributes of the dataset allow for an easy description of it
print(list(iris.keys()))

In [ ]:
print(iris['DESCR'])

In [ ]:
print('feature_names:\n', iris['feature_names'])
print()
print('target_names:\n', iris['target_names'])
print()
print('data:\n', iris['data'][:10]) # let's look for example at the first 10 samples
print()
print('target:\n', iris['target']) # notice that the dataset when loaded is ordered by labels (which we're not going to use)

In [ ]:
# extract data
X = iris.data

**Remark:** This should be an **unsupervised** learning setup. So, even though `iris.target` is present, we assume to have **no label** associated with the data.

Now let's see the shapes of the data.

In [ ]:
print('Shape of X:', X.shape)

(n, d) = X.shape
print('d:', d)
print('n:', n)

### 7.1.1 Data visualization

---


#### Histograms

Let's see the estimated pdf of each component (i.e., feature) by means of the histograms.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(18, 4))

for i in range(d):
    # a subplot for each feature
    plt.subplot(1, d, i+1)

    # histogram
    plt.hist(X[:, i], density=True, color=f'C{i}',alpha=0.8)

    # axis labels
    plt.xlabel('$x_{}$: {}'.format(i, iris.feature_names[i]), fontsize=12)
    if i == 0:
      plt.ylabel('estimated pdf', fontsize=12)

plt.show()

A couple of observations:

* the different ranges
* $x_2$ and $x_3$ are roughly bimodal, which might indicate the presence of an easy to spot cluster.


#### Scatter plots

We try to plot more features at the same time. We have 4 features but we can visualize at most 3D.


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(15, 6))

ax = fig.add_subplot(121, projection='3d')
ax.scatter(X[:, 0], X[:, 1], X[:, 2])
ax.set_xlabel(f'$x_0$: {iris.feature_names[0]}')
ax.set_ylabel(f'$x_1$: {iris.feature_names[1]}')
ax.set_zlabel(f'$x_2$: {iris.feature_names[2]}')

ax = fig.add_subplot(122, projection='3d')
ax.scatter(X[:, 0], X[:, 2], X[:, 3])
ax.set_xlabel(f'$x_0$: {iris.feature_names[0]}')
ax.set_ylabel(f'$x_2$: {iris.feature_names[2]}')
ax.set_zlabel(f'$x_3$: {iris.feature_names[3]}')

plt.tight_layout()
plt.show()

It seems that we can identify in a easier way clusters by watching $x_0, x_2$ and $x_3$ jointly instead of $x_0, x_1$ and $x_2$. Can't we?
Or maybe it just depends on where we are looking at the data from... Notice that in the next plots we're just changing `elev` and `azim` parameters for the subplots.

In [ ]:
fig = plt.figure(figsize=(15, 6))

ax = fig.add_subplot(121, projection='3d', elev=-150, azim=110)
ax.scatter(X[:, 0], X[:, 1], X[:, 2])
ax.set_xlabel(f'$x_0$: {iris.feature_names[0]}')
ax.set_ylabel(f'$x_1$: {iris.feature_names[1]}')
ax.set_zlabel(f'$x_2$: {iris.feature_names[2]}')

ax = fig.add_subplot(122, projection='3d', elev=-150, azim=110)
ax.scatter(X[:, 0], X[:, 2], X[:, 3])
ax.set_xlabel(f'$x_0$: {iris.feature_names[0]}')
ax.set_ylabel(f'$x_2$: {iris.feature_names[2]}')
ax.set_zlabel(f'$x_3$: {iris.feature_names[3]}')

plt.show()

Without the right perspective we may miss important clues.

2D plots are usually less subject to this kind of problems and clearer than 3D ones (personal opinion!). Let's try all possible combinations of pairs of features for the visualization.


In [ ]:
def plot_every_pair(X, colors=None, same_axis=False, label_pfx="x", include_var_names=True):
    d = X.shape[1]
    if colors is None:
        colors = np.zeros(X.shape[0])
    n_plots = d*(d-1)//2
    plt.figure(figsize=(3 * max(2, n_plots), 8))
    ct = 0
    for i in range(1, d+1):
        for j in range(i+1, d+1):
            ct += 1
            plt.subplot(2, int(np.ceil(n_plots/2)), ct)
            plt.scatter(X[:, i-1], X[:, j-1], c=colors)
            plt.xlabel(f"${label_pfx}_{i-1}$: {iris.feature_names[i-1] if include_var_names else ''}", fontsize=12)
            plt.ylabel(f"${label_pfx}_{j-1}$: {iris.feature_names[j-1] if include_var_names else ''}", fontsize=12)
            if same_axis:
                # Use same axis scaling
                plt.xlim([X.min()-1, X.max()+1])
                plt.ylim([X.min()-1, X.max()+1])
            plt.grid(alpha=0.5)
    plt.tight_layout()
    plt.show()

plot_every_pair(X)

Be careful about the different ranges!

In [ ]:
plot_every_pair(X, same_axis=True)

#### Seaborn

A cool package for data visualization is `seaborn`.

In [ ]:
import seaborn as sns
import pandas as pd

sns.pairplot(pd.DataFrame(X, columns=iris.feature_names)) # notice that the function expects the data to be a Pandas DataFrame
plt.show()

On the diagonal the pdf of the single features is shown, while in the other plots every pair of 2D scatter plot is shown. Notice that you can look only at half of it (upper or lower) and you'd get the same information, you can control this with the `corner=True` parameter. Several other visualizations are possible with this function. Check [its documentation](https://seaborn.pydata.org/generated/seaborn.pairplot.html).

__Note__ : the visualization above is rather difficult/cumbersome when the number of features is large.

### 7.1.2 Principal Component Analysis

Several ML problems involve thousands or even milions of features (think for example about some problems in genomics where you might have few samples (e.g., patients' tissue samples), and each one can have thousands of gene expression levels). This is an issue, particularly for some family of models, which is referred to as _the curse of dimensionality_. Furthermore, many curious and counter-intuitive (from our usual 2D or 3D intuitions) things happen in high dimensional spaces: for example, samples will tend to be far away from each other, making predictions and generalization tricky. As [Thomas Banchoff wrote](https://www.quantamagazine.org/a-mathematicians-guided-tour-through-high-dimensions-20210913/):

> “All of us are slaves to the prejudices of our own dimension."

However, there's hope: in most real-world problems, training instances are _not_ spread out uniformly across all dimensions; they tend to lie on a lower dimensional subspace (often referred to as _manifold_). Dimensionality reduction techniques aim to find such subspace. Several exist, but the most used and famous is __PCA__, which we're going to see in depth.

Recall the steps from the theory:

1. Let $X \in \mathbb{R}^{n \times d}$ be the dataset
1. Subtract the mean. Should we rescale?
  $$ X \leftarrow X - \overline{X} $$
1. Consider the sample covariance matrix
  $$\hat{\Sigma} = \frac{1}{n-1} X^\top X$$
1. Compute the symmetrical and semidefinite positive
  $$H = X^\top X$$
  and its eigen-decomposition
  $$ H = U \Lambda U^\top $$
  where $U \in \mathbb{R}^{d \times d}$ is the eigenvectors matrix and $\Lambda \in \mathbb{R}^{d \times d}$ is the eigenvalues matrix (diagonal).

  **Remark 1:** Eigenvalues and eigenvectors: $H \mathbf u = \lambda \mathbf u$

1. Now apply the transformation
  1. Lossless: apply $U^\top \mathbf x$ to each vector (simple rotation).
  2. Lossy:
    - Discard $l$ eigenvectors obtaining $\tilde{U} \in \mathbb{R}^{d \times d-l}$.
    - apply transformation $\tilde U^\top \mathbf x$ to each vector.

  To transform the entire dataset, simply do $XU$ or $X\tilde U$.



In [ ]:
X_mean = np.mean(X, axis=0, keepdims=True)
X0 = X - X_mean

H = (X0.T).dot(X0)
lam, U = np.linalg.eigh(H)

print("shapes:", lam.shape, U.shape)
print("eigenvalues:", lam)

We need to reverse `lam` and `U` since we want the eigenvalues to be sorted in descending order.

In [ ]:
# Sort the eigenvalues
lam = lam[::-1]
U = U[:, ::-1]

plt.plot(lam, 'o-')
plt.title("eigenvalues")
plt.grid()
plt.xlabel("component")
plt.show()

Now we apply the (lossless) transformation and plot the transformed data, notice how the first two principal components preserve the most variance in the data.

In [ ]:
# Apply rotation
X_rot = X0.dot(U)

sns.pairplot(pd.DataFrame(X_rot, columns=["$pc_0$","$pc_1$","$pc_2$","$pc_3$"]), corner=True)
plt.show()

We consider only the first two eigenvectors and discard the other two, effectively reduced the number of dimensions from 4 to 2. Notice that we lose some interpretability in doing so. What are the new features now?

In [ ]:
# Apply reduced transformation
l = 2  # columns to discard
Utilde = U[:, :d-l]
X_red = X0.dot(Utilde)
# Equivalent to X_red = X_rot[:, :d-l]

plot_every_pair(X_red, same_axis=True, label_pfx="pc", include_var_names=False)

As usual, `sklearn` can speed up our work! Notice that the result is equal.

In [ ]:
# PCA with sklearn
from sklearn.decomposition import PCA

# d:    num of original features (= num of all principal components)
# l:    num of discarded principal components
# d-l:  num of considered principal components
pca = PCA(n_components=d-l)
pca.fit(X0)
X_red = pca.transform(X0)
X_red[:, 0] = -X_red[:, 0]  # reverse dimension zero since corresponding eigenvector is symmetric wrt our solution

plot_every_pair(X_red, same_axis=True, label_pfx="pc", include_var_names=False)

### 7.1.3 Data reconstruction

What about if we want to come back to the original number of components?

$$\mathbf x \to \mathbf {\tilde x} \to \mathbf x_{rec} \approx \mathbf x$$

where $\mathbf x \in \mathbb{R}^{d}, \mathbf {\tilde x} \in \mathbb{R}^{d-l}$ and $\mathbf x_{rec} \in \mathbb{R}^{d}$. In this way we are able to **compress** the original data in a low dimensional space and restore them (in a **lossy** way!).

- Transformation: $\mathbf{\tilde x}=\tilde U^\top \mathbf x$.
- Reconstruction (inverse transformation): $\mathbf x_{rec} = \tilde U \mathbf{\tilde x}$.

In [ ]:
# Visualize original vs reconstructed dataset
fig = plt.figure(figsize=(18, 4))
fig.subplots_adjust(wspace=.4)

# Original dataset
ax = fig.add_subplot(131, projection='3d', elev=30, azim=160)

ax.scatter(X0[:, 0], X0[:, 2], X0[:, 3]) #, X[:, 3])
ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_2$')
ax.set_zlabel('$x_3$')
ax.set_title("$X$")

# Principal components
ax = fig.add_subplot(132)
ax.scatter(X_red[:, 0], X_red[:, 1])
ax.set_xlabel('$pc_0$')
ax.set_ylabel('$pc_1$')
ax.set_title("Principal Components")
ax.axis("equal")

# Reconstructed dataset
ax = fig.add_subplot(133, projection='3d', elev=30, azim=160)
# reconstruct, notice the inverse transformation is taken care directly by sklearn method!
X_rec = pca.inverse_transform(X_red)
# which is equivalent to
# X_red_ = X.dot(Utilde)
# X_rec_ = X_red_.dot(Utilde.T)

ax.scatter(X_rec[:, 0], X_rec[:, 2], X_rec[:, 3])
ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_2$')
ax.set_zlabel('$x_3$')
ax.set_title("$X$ reconstructed")
plt.show()

In [ ]:
print("Original Data")
sns.pairplot(pd.DataFrame(X, columns=iris.feature_names), corner=True)
plt.show()
print("\n\nReduced Data")
sns.pairplot(pd.DataFrame(X_red, columns=[f'$pc_{pc}$' for pc in range(d-l)]), corner=True)
plt.show()
print("\n\nReconstructed Data")
sns.pairplot(pd.DataFrame(X_rec, columns=iris.feature_names), corner=True)
plt.show()

Indeed we can see a loss of information due to the compression the data have undergone. We can also get a sense in terms of explained variance of _how much_ information we lost in the process:

In [ ]:
pca.explained_variance_ratio_

The first two principal components account for more or less 98% of the variance in the original data. Effectively we halved the features, but only lost 2% of the implied variance. We will see in the last part of this lab a clearer example of how much is lost in this process of compression and reconstruction.

### 7.1.4 Clustering: k-means

Now that we managed to represent the original dataset in a low dimensional space we can use the $k$-means clustering technique to classify the data into groups. Sklearn, as usual, has a practical [KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) class ready to use.

In [ ]:
from sklearn.cluster import KMeans

k_clusters = 2

k_means = KMeans(n_clusters=k_clusters, n_init=10)
cluster_label = k_means.fit_predict(X_red)

In [ ]:
# 3d
fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot(121, projection='3d', elev=30, azim=160)
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=cluster_label)
ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_1$')
ax.set_zlabel('$x_2$')
# 2d PC
ax = fig.add_subplot(122)
ax.scatter(X_red[:, 0], X_red[:, 1], c=cluster_label)
ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_1$')
ax.axis("equal")
plt.show()

Since we know that there are three classes in `iris.target`...

In [ ]:
# Plot the real labels
# 3d
fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot(121, projection='3d', elev=30, azim=160)
for c, label in enumerate(iris.target_names):
  idxs = iris.target == c
  ax.scatter(X[idxs, 0], X[idxs, 1], X[idxs, 2], label=label)
ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_1$')
ax.set_zlabel('$x_2$')

# 2d PC
ax = fig.add_subplot(122)
for c, label in enumerate(iris.target_names):
  idxs = iris.target == c
  ax.scatter(X_red[idxs, 0], X_red[idxs, 1], label=label)
ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_1$')
ax.axis("equal")
ax.legend()
ax.set_title("Classes (not clusters!)")
plt.show()


However, k-means (as well as any other clustering method) does not necessarily retrieve the same classes, because classes are not necessarily confined into clusters.

In [ ]:
from sklearn.cluster import KMeans

k_clusters = 3

k_means = KMeans(n_clusters=k_clusters, n_init=10)
cluster_label = k_means.fit_predict(X_red)

# 3d
fig = plt.figure(figsize=(16, 4))
ax = fig.add_subplot(131, projection='3d', elev=30, azim=160)
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=cluster_label)
ax.set_xlabel(r'$x_0$')
ax.set_ylabel(r'$x_1$')
ax.set_zlabel(r'$x_2$')
# 2d PC
ax = fig.add_subplot(132)
ax.scatter(X_red[:, 0], X_red[:, 1], c=cluster_label)
ax.set_xlabel(r'$x_0$')
ax.set_ylabel(r'$x_1$')
ax.axis("equal")
ax.set_title("clusters")

# classes
ax = fig.add_subplot(133)
ax.scatter(X_red[:, 0], X_red[:, 1], c=iris.target)
ax.set_xlabel(r'$x_0$')
ax.set_ylabel(r'$x_1$')
ax.axis("equal")
ax.set_title("classes")

plt.show()

Overall not a bad result considering the two classes are quite close to one another. Don't let the difference in color fool you, the encoding of the cluster might be different from the encoding of the class. The important thing is the separation.

#### Final considerations

- We can cross-validate the number of clusters ([silhouette](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html))
- Variety of clustering methods with different behaviours ([comparison](https://scikit-learn.org/stable/modules/clustering.html#clustering))



## 7.2 Image compression with Fashion-MNIST

---

In this part, we will see how we can compress an image dataset, reducing the number of components needed to represent each image. For this purpose we consider the dataset Fashion-MNIST provided by Zalando.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

fashion_mnist = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)

classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

print('Dataset has {} instances'.format(len(fashion_mnist)))
print(f'Shape of images: {fashion_mnist.data.shape[1:]}')

X = fashion_mnist.data.numpy()
y = fashion_mnist.targets.numpy()

In [ ]:
# Let's plot some samples to get a general idea of the dataset
np.random.seed(42)

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X[i].reshape(28, 28), cmap="binary")
    plt.xlabel(classes[y[i]])
plt.show()

Vectorize the images.

In [ ]:
n_samples = X.shape[0]
X, y = X[:n_samples], y[:n_samples]
print("X shape:", X.shape)

# Reshape to vectors and rescale to [0, 1]
w, h = X.shape[1:3]
X_vec = X.reshape(-1, w * h) /255.
print("X_vec:", X_vec.shape)

Let's plot the principal components.

In [ ]:
# PCA
n_components = 200

X_mean = X_vec.mean(axis=0, keepdims=True)
X0 = X_vec - X_mean
pca = PCA(n_components=n_components)
pca.fit(X0)

plt.figure()
plt.title('eigenvalues')
plt.plot(pca.singular_values_**2)
plt.show()

In [ ]:
# compress
X_red = pca.transform(X0)
# extract
X_rec = pca.inverse_transform(X_red)
X_rec += X_mean

# reshape to image size and range
x_image_rec = 255 * X_rec.clip(0, 1).reshape(-1, w, h)
x_image_orig = 255 * X.clip(0, 1).reshape(-1, w, h)

In [ ]:
# draw some random images
p = np.random.choice(X.shape[0], size=5)

plt.figure(figsize=(7,10))
for i in range(0,len(p)*2 -1,2):
    plt.subplot(len(p),2,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.title("Original")
    plt.imshow(X[p[i//2]].reshape(28, 28), cmap="binary")
    plt.subplot(len(p),2,i+2)
    plt.title("Reconstruction")
    plt.imshow(x_image_rec[p[i//2]].reshape(28, 28), cmap="binary")
plt.tight_layout()
plt.show()


It seems that we are able to decently rebuild the original images using way less features! The object seem still to be recognizable. Now what happens if we add noise to the original images?

### Image denoising

In [ ]:
# Add noise to X
X_noisy = X_vec + np.random.randn(*X_vec.shape)*.15

# PCA
n_components = 75 # let's use even less principal components this time
X_mean = X_noisy.mean(axis=0, keepdims=True)
X0 = X_noisy - X_mean
pca = PCA(n_components=n_components)
pca.fit(X0)

plt.figure()
plt.title('eigenvalues')
plt.plot(pca.singular_values_**2)
plt.show()

# compress
X_red = pca.transform(X0)
# extract
X_rec = pca.inverse_transform(X_red)
X_rec += X_mean

# reshape to image size and range
x_image_orig = 255 * X.clip(0, 1).reshape(-1, w, h)
x_image_noisy = 255 * X_noisy.clip(0, 1).reshape(-1, w, h)
x_image_rec = 255 * X_rec.clip(0, 1).reshape(-1, w, h)

# draw some random images
np.random.seed(42)
p = np.random.choice(X.shape[0], size=5)

plt.figure(figsize=(7,10))
for i in range(0,len(p)*3 -1,3):
    plt.subplot(len(p),3,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.title("Original")
    plt.imshow(X[p[i//3]].reshape(28, 28), cmap="binary")
    plt.subplot(len(p),3,i+2)
    plt.title("Noisy")
    plt.imshow(x_image_noisy[p[i//3]].reshape(28, 28), cmap="binary")
    plt.subplot(len(p),3,i+3)
    plt.title("Reconstruction")
    plt.imshow(x_image_rec[p[i//3]].reshape(28, 28), cmap="binary")
plt.tight_layout()
plt.show()

Notice that the noise introduced is mostly gone and only useful features are preserved.

Now let's see if using only two dimensions we are able to plot the dataset in a clusterized fashion.

In [ ]:
x_dim, y_dim = 0, 1 # we use the first two components of PCA
plt.figure(figsize=(16, 12))
for d in range(10):
    ii = np.where(y==d)[0]
    plt.scatter(X_red[ii][:, x_dim], X_red[ii][:, y_dim], marker=f"${d}$", label=d)
plt.xlabel("$pc_0$",fontsize=12)
plt.ylabel("$pc_1$",fontsize=12)
plt.legend(labels=classes)
plt.show()

Not bad, overall we can see that even with only two features there's some separation between classes.

__Advanced note:__ Several other methods are commonly used in combination with PCA to obtain visualization of high dimensional datasets. A commonly used one is [TSNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html), also implemented in sklearn. The following snippet of code is an example of how it's used (it can take up to 5 minutes to run it).

In [ ]:
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import matplotlib as mpl


def plot_digits(X, y, min_distance=0.05, figsize=(13, 10)):
    """Just a helper function for the visualization, no need to understand it"""
    X_normalized = MinMaxScaler().fit_transform(X)
    neighbors = np.array([[10., 10.]])
    plt.figure(figsize=figsize)
    cmap = mpl.cm.get_cmap("prism")
    digits = np.unique(y)
    for digit in digits:
        plt.scatter(X_normalized[y == digit, 0], X_normalized[y == digit, 1], c=[cmap(digit / 9)])
    plt.axis("off")
    ax = plt.gcf().gca()  # get current axes in current figure
    for index, image_coord in enumerate(X_normalized):
        closest_distance = np.linalg.norm(np.array(neighbors) - image_coord, axis=1).min()
        if closest_distance > min_distance:
            neighbors = np.r_[neighbors, [image_coord]]
            plt.text(image_coord[0], image_coord[1], str(int(y[index])),
                         color=cmap(y[index] / 9), fontdict={"weight": "bold", "size": 16})

pca_tsne = Pipeline([
    ("pca", PCA(n_components=0.9, random_state=42)),
    ("tsne", TSNE(n_components=2, random_state=42)),
])
X_pca_tsne_reduced = pca_tsne.fit_transform(X0)
plot_digits(X_pca_tsne_reduced, y)
plt.show()
